## Setup

In [13]:
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random

## Input data directory
data_dir = "facultymanual"
inputdirectory = Path(f"./data_input/{data_dir}")
## This is where the output csv files will be written
out_dir = data_dir
outputdirectory = Path(f"./data_output/{out_dir}")

## Load Documents

In [14]:
## Dir PDF Loader
# loader = PyPDFDirectoryLoader(inputdirectory)
## File Loader
# loader = PyPDFLoader("./data/MedicalDocuments/orf-path_health-n1.pdf")
loader = DirectoryLoader(inputdirectory, show_progress=True)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))
print(pages[3].page_content)


100%|██████████| 1/1 [00:00<00:00,  1.50it/s]

Number of chunks =  279
ARLENE O.  TRILLANES, PHD Vice President for Operations, NU Manila Revision Committee 13 December 2023

Reviewed : Endorsed :

AIMEE JANE T. MARTINEZ Assistant Vice President  for Human Resource

JOHANNA G. MINGLANA , LPT Senior Director Quality Management Office 13 December 2023  4 January 2024

Concurred : Approved:

ROSAURO E . MANUEL, DBA Group Academic Of ficer  / Chief Information Officer

RENATO CARLOS H. ERMITA, JR., PHD President  & CEO 5 January 2024  8 January 2024

Title: FACULTY  MANUAL  Reference No. OPR – HRD – D – M – 003 DEPARTMENT  HUMAN RESOURCES This is a controlled document. Photocopying or printing of this document, without the controlled mark, shall make this document an uncontrolled copy.

Date of Effectivity  1 August  2023 Control Mark QMO Reviewed  Page 5 of 147 Supersedes  1 December 2019

DISTRIBUTION LIST  COPY  REMARKS Quality Management Office  Original  Hard Copy & Electronic Copy Office of the President  Copy 1  Electronic Copy 

## Create a dataframe of all the chunks

In [15]:
from helpers.df_helpers import documents2Dataframe
df = documents2Dataframe(pages)
print(df.shape)
df.head()

(279, 3)


,text,source,chunk_id
0,Title: FACULTY MANUAL Reference No. OPR – HR...,data_input/facultymanual/facultymanual.txt,6a376de831c34598874d5d3a9b6f6a8b
1,Date of Effectivity 1 August 2023 Control Ma...,data_input/facultymanual/facultymanual.txt,940095c9ece34aa7b95e81a5f6176577
2,Coordinated with the Faculty Services Office (...,data_input/facultymanual/facultymanual.txt,51be91c9a2804056b1c69d409c9450fd
3,"ARLENE O. TRILLANES, PHD Vice President for O...",data_input/facultymanual/facultymanual.txt,c84946cf9fa742cf9adebfc16b298376
4,MANUAL VERSION CONTROL LOG .....................,data_input/facultymanual/facultymanual.txt,3ff2da00d90b4c3f854c0cf56a41c905


## Extract Concepts

In [16]:
## This function uses the helpers/prompt function to extract concepts from text
from helpers.df_helpers import df2Graph
from helpers.df_helpers import graph2Df

If regenerate is set to True then the dataframes are regenerated and Both the dataframes are written in the csv format so we dont have to calculate them again. 

        dfne = dataframe of edges

        df = dataframe of chunks


Else the dataframes are read from the output directory

In [17]:
## To regenerate the graph with LLM, set this to True
regenerate = True

if regenerate:
    concepts_list = df2Graph(df, model='zephyr:latest')
    dfg1 = graph2Df(concepts_list)
    if not os.path.exists(outputdirectory):
        os.makedirs(outputdirectory)
    
    dfg1.to_csv(outputdirectory/"graph.csv", sep="|", index=False)
    df.to_csv(outputdirectory/"chunks.csv", sep="|", index=False)
else:
    dfg1 = pd.read_csv(outputdirectory/"graph.csv", sep="|")

dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4 
## Increasing the weight of the relation to 4. 
## We will assign the weight of 1 when later the contextual proximity will be calculated.  
print(dfg1.shape)
dfg1.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data_output/facultymanual/graph.csv'

## Calculating contextual proximity

In [6]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # Self join with chunk id as the key will create a link between terms occuring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(dfg1)
dfg2.tail()

,node_1,node_2,chunk_id,count,edge
2827,world-class health facilities,nhm strategies,"0857ab4513ad4383aed095bcf24506fa,0857ab4513ad4...",10,contextual proximity
2828,world-class health facilities,rural areas,"0857ab4513ad4383aed095bcf24506fa,0857ab4513ad4...",2,contextual proximity
2829,world-class health facilities,social norms,"0857ab4513ad4383aed095bcf24506fa,0857ab4513ad4...",2,contextual proximity
2830,world-class health facilities,urban areas,"0857ab4513ad4383aed095bcf24506fa,0857ab4513ad4...",2,contextual proximity
2831,world-class health facilities,urban slums,"0857ab4513ad4383aed095bcf24506fa,0857ab4513ad4...",2,contextual proximity


### Merge both the dataframes

In [7]:
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({"chunk_id": ",".join, "edge": ','.join, 'count': 'sum'})
    .reset_index()
)
dfg

,node_1,node_2,chunk_id,edge,count
0,56 articles,extensive literature search,"d7a3e5085c7f4de4bc28fb0bd9cb0a94,d7a3e5085c7f4...",contextual proximity,2
1,[54],increasing violence against healthcare personnel,"640835e2521045a395ab6465cc1ba4ca,640835e252104...",contextual proximity,2
2,[55],increasing violence against healthcare personnel,"640835e2521045a395ab6465cc1ba4ca,640835e252104...",contextual proximity,2
3,a bad situation,increasing violence against healthcare personnel,"640835e2521045a395ab6465cc1ba4ca,640835e252104...",contextual proximity,2
4,a worrisome new trend,increasing violence against healthcare personnel,"640835e2521045a395ab6465cc1ba4ca,640835e252104...",contextual proximity,2
...,...,...,...,...,...
753,world-class health facilities,nhm strategies,"0857ab4513ad4383aed095bcf24506fa,0857ab4513ad4...",contextual proximity,10
754,world-class health facilities,rural areas,"0857ab4513ad4383aed095bcf24506fa,0857ab4513ad4...",contextual proximity,2
755,world-class health facilities,social norms,"0857ab4513ad4383aed095bcf24506fa,0857ab4513ad4...",contextual proximity,2
756,world-class health facilities,urban areas,"0857ab4513ad4383aed095bcf24506fa,0857ab4513ad4...",contextual proximity,2


## Calculate the NetworkX Graph

In [8]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(215,)

In [9]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

### Calculate communities for coloring the nodes

In [10]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

Number of Communities =  17
[['56 articles', 'analysis', "corresponding authors' experiential knowledge", 'extensive literature search', 'peer-reviewed journals'], ['[54]', '[55]', 'a bad situation', 'a worrisome new trend', 'adequately compensated', 'can reverse the situation', 'defensive medicine practices', 'increasing violence against healthcare personnel', 'intense focus on specialization', 'low physician-to-patient ratio', 'overwhelmed physicians', 'primary care physicians', 'private marketplace', 'protect themselves by ordering unnecessary tests and procedures', 'results in delays in attending patients', 'set in', 'tempted to take on more patients than they can reasonably serve', 'thoughtful approach to government planning', 'underpaid physicians', 'unethical practices by pharmaceutical companies', 'will not be able to solve this'], ['accredit health facilities', 'enforcement of existing rules', 'health insurance scheme for central government employees', 'health system standardi

### Create a dataframe for community colors

In [11]:
import seaborn as sns
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)
colors

,node,color,group
0,56 articles,#db577e,1
1,analysis,#db577e,1
2,corresponding authors' experiential knowledge,#db577e,1
3,extensive literature search,#db577e,1
4,peer-reviewed journals,#db577e,1
...,...,...,...
210,rural medical assistants (rmas),#9ddb57,15
211,limited uptake,#6edb57,16
212,national health protection mission,#6edb57,16
213,private health sector systems,#575edb,17


### Add colors to the graph

In [12]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [12]:
from pyvis.network import Network

graph_output_directory = "./docs/index.html"

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])

net.show(graph_output_directory, notebook=False)

./docs/index.html
